# import and set up

In [1]:
# 这个notebook是用来把设计表里平铺的数据，散落/更新/添加/修改到项目的配置表里
# 这张表格是设计表： intermediateTables\PreCATFishesOrigin.xlsx
# 我们要使用它，来向 configs\xlsx_channel\2001\fish_env_affinity.xlsx 以及 configs\xlsx_channel\2001\fish_pond.xlsx 中添加/修改数据
# 数据应该是单向流动的，尽量不要修改configs目录下的表格
import os.path
from openpyxl.styles.fonts import Font

rootPath = ".."
OriginWorkbookPath = r"intermediateTables\PreCATFishesOriginV2.xlsx"
AffinityWorkbookPath = r"configs\xlsx_channel\2001\fish_env_affinity.xlsx"
FishBasicWorkbookPath = r"configs\xlsx_channel\2001\fish_basic.xlsx"
PondWorkbookPath = r"configs\xlsx_channel\2001\fish_pond.xlsx"
PriceToolWorkbookPath = r"intermediateTables\鱼单价工具.xlsx"

OriginWorkbookPath = os.path.join(rootPath, OriginWorkbookPath)
AffinityWorkbookPath = os.path.join(rootPath, AffinityWorkbookPath)
PondWorkbookPath = os.path.join(rootPath, PondWorkbookPath)
FishBasicWorkbookPath = os.path.join(rootPath, FishBasicWorkbookPath)
PriceToolWorkbookPath = os.path.join(rootPath, PriceToolWorkbookPath)

sheetOrigin = "鱼种设计表"
sheetDictQuality = "品质字典"

startColumnNameStruct = "|调整后遮蔽"
startColumnNameBait = "|饵"
startColumnNameWaterLayer = "|觅食层"
startColumnNameTemp = "|温度"

columnNameFishSpecies = "Species"

dictTempColumns = {
    "中间温":"temperature_fav",
    "调整后温度耐受":"temp_affected_ratio",
    "温度耐受阈值":"temp_threshold",
    }

In [2]:
import pandas as pd

OriginDf = pd.read_excel(OriginWorkbookPath, sheet_name="鱼种设计表")
# OriginDf.head()
OriginDf.columns

dictQualityDf = pd.read_excel(OriginWorkbookPath, sheet_name=sheetDictQuality)
WeightParamDf = pd.read_excel(OriginWorkbookPath, sheet_name="中英文和LW参数")
priceDf = pd.read_excel(PriceToolWorkbookPath, sheet_name="鱼单价工具")

In [3]:
# 把columns切成几段，以那些|开头的为分割线
def split_columns(columns):
    split_columns = []
    current_segment = []
    for col in columns:
        # 强制转为字符串再判断前缀，避免非 str 类型（如 int）报错
        if str(col).startswith("|"):
            if current_segment:
                split_columns.append(current_segment)
            current_segment = [col]
        else:
            current_segment.append(col)
    if current_segment:
        split_columns.append(current_segment)
    return split_columns

# print the segments
splittedCols = split_columns(OriginDf.columns)
print(splittedCols)

for segment in splittedCols:
    print(segment)

[['Species', '鱼种', '是否保底', '等阶', '细化等阶', '备注', '本阶内单条价格', '品质', '地图', '阶内权重', 'xp参考', 'gold参考'], ['|温度', '舒适温度min', '舒适温度max', '中间温', '温度耐受', '温度耐受调整', '调整后温度耐受', '温度耐受阈值'], ['|遮蔽', '水下结构体]开放水域', '水下结构体]水草', '水下结构体]石头', '水下结构体]沉木', '水下结构体]桥墩'], ['|调整后遮蔽', '[水下结构体]开放水域', '[水下结构体]水草', '[水下结构体]石头', '[水下结构体]沉木', '[水下结构体]桥墩', '[水下结构体]深坑', '[水下结构体]尖脊', '[水下结构体]断层', '[水下结构体]岩架', '[水下结构体]湾子', '[水下结构体]泥底', '[水下结构体]碎石底'], ['|觅食层', '[地图水层]表层', '[地图水层]中层', '[地图水层]底层'], ['|真饵', '[真饵]种子', '[真饵]昆虫', '[真饵]甲壳', '[真饵]鱼饵', '[真饵]鱼卵', '[真饵]面团', '[真饵]谷物', '[真饵]肉饵', '[真饵]乳制'], ['|拟饵', '[拟饵]T尾', '[拟饵]卷尾', '[拟饵]软虫', '[拟饵]虾管', '[拟饵]米诺', '[拟饵]波爬', '[拟饵]勺子亮片', '[拟饵]旋转亮片', '[拟饵]VIB', '[拟饵]水面拖拉机', '[拟饵]铅笔', '[拟饵]嘈杂饵', '[拟饵]多节鱼', '[拟饵]雷蛙', '[拟饵]胡须佬', '[拟饵]复合亮片', '[拟饵]摇滚饵'], ['|饵长', 'min_adapt_lure_ratio', 'max_adapt_lure_ratio', 'under_length_decay_coeff', 'over_length_decay_coeff', 'max_accept_length_ratio'], ['|其他', '气压敏感度', '时段习性组']]
['Species', '鱼种', '是否保底', '等阶', '细化等阶', '备注', '本阶内单条价格', '品质', '地图', '阶内权重', 'xp

In [4]:
# 从splittedCols里取出以startColumnNameStruct开头的segment,并去除startColumnNameStruct这一列
# 然后动态的遍历修改AffinitySpreadSheet中的数据
def get_segment_by_start_name(startName):
    for segment in splittedCols:
        if segment[0].startswith(startName):
            return segment[1:]
    return None

# 取出以|遮蔽开头的segment,并去除|遮蔽这一列
segmentStruct = get_segment_by_start_name(startColumnNameStruct)
print(segmentStruct)


['[水下结构体]开放水域', '[水下结构体]水草', '[水下结构体]石头', '[水下结构体]沉木', '[水下结构体]桥墩', '[水下结构体]深坑', '[水下结构体]尖脊', '[水下结构体]断层', '[水下结构体]岩架', '[水下结构体]湾子', '[水下结构体]泥底', '[水下结构体]碎石底']


In [5]:
# 开始遍历修改AffinitySpreadSheet中的数据
import openpyxl

openpyxl.open(AffinityWorkbookPath, read_only=False, keep_links=False)
AffinityPyxl = openpyxl.load_workbook(AffinityWorkbookPath)
openpyxl.open(FishBasicWorkbookPath, read_only=False, keep_links=False)
FishBasicPyxl = openpyxl.load_workbook(FishBasicWorkbookPath)
# 温度sheet的名字是“TempAffinity”
tempAffinitySheetPath = AffinityPyxl["TempAffinity"]
structAffinitySheetPath = AffinityPyxl["StructAffinity"]
baitTypeAffinitySheetPath = AffinityPyxl["BaitTypeAffinity"]
topAffinitySheetPath = AffinityPyxl["FishEnvAffinity"]

topAffinitySheet = AffinityPyxl["FishEnvAffinity"]

fishQualitySheet = FishBasicPyxl["BasicFishQuality"]

fishNames = OriginDf[columnNameFishSpecies].tolist()
originRows = OriginDf.shape[0]


In [6]:
# fishQuality表的第二行是列名，第五行起是数据
# 先找到id为name，name_language， art_id， model_id， species， size_category， volume_exponent， mass_factor的几列的索引
id_col = None
fish_quality_name_col = None
name_language_col = None
art_id_col = None
model_id_col = None
species_col = None
size_category_col = None
volume_exponent_col = None
mass_factor_col = None
for col in range(1, fishQualitySheet.max_column + 1):
    cell = fishQualitySheet.cell(row=2, column=col)
    if cell.value == "id":
        id_col = col
    elif cell.value == "name":
        fish_quality_name_col = col
    elif cell.value == "name_language":
        name_language_col = col
    elif cell.value == "art_id":
        art_id_col = col
    elif cell.value == "model_id":
        model_id_col = col
    elif cell.value == "species":
        species_col = col
    elif cell.value == "size_category":
        size_category_col = col
    elif cell.value == "volume_exponent":
        volume_exponent_col = col
    elif cell.value == "mass_factor":
        mass_factor_col = col

In [7]:
# find the column index from target worksheet, row 2 is the header
def find_column_index(sheet, header):
    for col in range(1, sheet.max_column + 1):
        cell = sheet.cell(row=2, column=col)
        if cell.value == header:
            return col
    return None

def findRowByNameFromWolongSheet(sheet, name):
    column_index = find_column_index(sheet, "name")

    for row in range(5, sheet.max_row + 1):
        cell = sheet.cell(row=row, column=2)
        if cell.value == name:
            return row
    return None


# 在fish quality、fish env affinity表里找到对应的条目，或新建条目

In [8]:
# 创建列名映射字典
COLUMN_MAPPING = {
    "min_adapt_lure_ratio": "min_adapt_lure_ratio",
    "max_adapt_lure_ratio": "max_adapt_lure_ratio",
    "under_length_decay_coeff": "under_length_decay_coeff",
    "over_length_decay_coeff": "over_length_decay_coeff",
    "max_accept_length_ratio": "max_accept_length_ratio",
    "气压敏感度": "pressure_sensitivity"  # 特殊处理中文列名
}

In [9]:

def getFishNameRowIdxFromTopAffinitySheet(fishName,quality) -> int:
    '''
    fishName和quality输入进入函数，返回在fish_env_affinity.xlsx表格中找到的行号
    如果没有找到，就在表格中添加一行数据，并返回这个新行的行号，表格中这加入的新行数据的字体颜色是绿色
    '''
    qualityStr = dictQualityDf[dictQualityDf["品质"] == quality]["后缀"].values[0]
    prefixedFishSpeciesName = f"Fish_{fishName}"
    prefixedFishQualityName = f"Fish_{fishName}{qualityStr}"
    fishQualityName = f"{fishName}{qualityStr}"
    # 在表格中查找这个名字
    foundFishName = False
    # 在表格中B列查找这个名字
    for row in range(5, topAffinitySheet.max_row + 1):
        cell = topAffinitySheet.cell(row=row, column=2)
        if cell.value == prefixedFishQualityName:
            foundFishName = True
            return row
            break
    if not foundFishName:
        lastId = topAffinitySheet.cell(row=topAffinitySheet.max_row, column=1).value
        if lastId is None:
            lastId = 1010300
        else:
            newId = lastId + 10 - lastId % 10

        # add a new row, and set the fish name in B column
        new_row_idx = topAffinitySheet.max_row + 1
        topAffinitySheet.cell(row=new_row_idx, column=2, value=prefixedFishQualityName)
        topAffinitySheet.cell(row=new_row_idx, column=1, value=newId)
        # set the fish name in the 3rd column
        topAffinitySheet.cell(row=new_row_idx, column=3, value=fishQualityName)
        # from column 4, copy the values from the last row, and set this row font color to green
        for col in range(4, topAffinitySheet.max_column + 1):
            cell = topAffinitySheet.cell(row=new_row_idx, column=col)
            new_cell = topAffinitySheet.cell(row=new_row_idx, column=col)
            new_cell.value = cell.value
            new_cell.font = openpyxl.styles.Font(color="990099")

        # 检查fish_basic.xlsx中的fish_quality工作表中是否有鱼的这个品质配置，如果没有，进行相应的配置，并把相应的行设置成绿色
        foundQuality = False
        # fishQuality表的第二行是列名，第五行起是数据
        for rowIdx in range(5, fishQualitySheet.max_row + 1):
            cell = fishQualitySheet.cell(row=rowIdx, column=fish_quality_name_col)
            if cell.value == fishName:
                foundQuality = True
                break
        if not foundQuality:
            lastRowIdx = fishQualitySheet.max_row
            lastId = fishQualitySheet.cell(row=lastRowIdx, column=id_col).value
            print(f"lastId: {lastId}")
            newId = lastId + 10 - lastId % 10
            print(f"newId: {newId}")
            # 在fishQualitySheet中添加一行数据
            # create a new row by copying the last row
            new_row_idx = lastRowIdx + 1
            for col in range(1, fishQualitySheet.max_column + 1):
                cell = fishQualitySheet.cell(row=lastRowIdx, column=col)
                new_cell = fishQualitySheet.cell(row=new_row_idx, column=col)
                new_cell.value = cell.value
                new_cell.font = openpyxl.styles.Font(color="990099")
            # set the new row's id to newId
            fishQualitySheet.cell(row=new_row_idx, column=id_col, value=newId)
            # set the new row's name to fishName
            fishQualitySheet.cell(row=new_row_idx, column=fish_quality_name_col, value=fishQualityName)
            # lower case of prefixedFishSpeciesName
            lowerCasedPrefixedSpecies = prefixedFishSpeciesName.lower()
            # set the new row's name_language to name_language_value
            fishQualitySheet.cell(row=new_row_idx, column=name_language_col, value=lowerCasedPrefixedSpecies)
            # art_id
            fishQualitySheet.cell(row=new_row_idx, column=art_id_col, value=lowerCasedPrefixedSpecies+"_2D")
            fishQualitySheet.cell(row=new_row_idx, column=model_id_col, value=lowerCasedPrefixedSpecies)
            fishQualitySheet.cell(row=new_row_idx, column=species_col, value=lowerCasedPrefixedSpecies)
            sizeCategoryStr = dictQualityDf[dictQualityDf["品质"] == quality]["枚举"].values[0]
            fishQualitySheet.cell(row=new_row_idx, column=size_category_col, value=sizeCategoryStr)
            # volume_exponent
            try:
                volumeExponentValue = WeightParamDf[WeightParamDf["手调鱼种string"] == fishName]["重量参数b"].iloc[0]
            except IndexError:
                print(f"警告: 鱼种 '{fishName}' 的重量参数b未找到，使用默认值3.0")
                volumeExponentValue = 1.0
                
            # mass_factor
            try:
                massFactorValue = WeightParamDf[WeightParamDf["手调鱼种string"] == fishName]["重量参数a"].iloc[0]
            except IndexError:
                print(f"警告: 鱼种 '{fishName}' 的重量参数a未找到，使用默认值0.0127")
                massFactorValue = 1.0

        return new_row_idx
    return -1
        
for baitAffRow in range(originRows):
    fishName = OriginDf.iloc[baitAffRow][columnNameFishSpecies]
    # 强制把品质列转成字符串再 split，避免 int 没有 split
    raw_q = OriginDf.at[baitAffRow, "品质"]
    qualities = str(raw_q).split("、")
    for q in qualities:
        # 如果后续需要用整数作 key，再尝试转回 int
        try:
            quality_key = int(q)
        except ValueError:
            quality_key = q
        getFishNameRowIdxFromTopAffinitySheet(fishName, quality_key)

In [10]:
import string


def extract_qualities_for_fish(fish_name, fish_col=columnNameFishSpecies, quality_col="品质"):
    """根据鱼名从origin设计表中提取品质列表，转换为整数或保留原字符串"""
    # 查找对应行
    matches = OriginDf[OriginDf[fish_col] == fish_name]
    if matches.empty:
        return []
    raw_q = matches.iloc[0][quality_col]
    # 分割并转换
    qualities = []
    for q in str(raw_q).split("、"):
        try:
            qualities.append(int(q))
        except ValueError:
            qualities.append(q)
    return qualities

def get_quality_strings_for_fishname(fish_name, fish_col=columnNameFishSpecies, quality_col="品质"):
    # dictQualityDf

    """根据鱼名从origin设计表中提取品质列表，转换为整数或保留原字符串"""
    # 查找对应行
    matches = OriginDf[OriginDf[fish_col] == fish_name]
    if matches.empty:
        return []
    raw_q = matches.iloc[0][quality_col]
    # 分割并转换
   
    qualityStrings = []
    for q in str(raw_q).split("、"):
        postFix = dictQualityDf[dictQualityDf["品质"] == int(q)].iloc[0]["后缀"]
        qualityStrings.append(postFix)

    return  qualityStrings

def get_qualitystr_from_qualityidx(qualityidx:int)->string:
    return dictQualityDf[dictQualityDf["品质"] == int(qualityidx)].iloc[0]["后缀"]

# 水下结构体数据格式转换
将源表中的宽格式数据（每行一种鱼有多列不同结构体的数值）转换成目标表中的结构化格式（每行一种鱼，结构体类型和系数交替排列的多列）

In [11]:
import pandas as pd
from openpyxl.utils.dataframe import dataframe_to_rows

# 定义宽格式转结构化格式的函数
def convert_wide_to_structured(source_df, struct_columns):
    """
    将源表中的宽格式数据转换为目标表中的结构化格式
    
    参数:
    - source_df: 源DataFrame，包含鱼类和结构体系数
    - struct_columns: 水下结构体列名列表
    
    返回:
    - 转换后的DataFrame，适合写入目标表格
    """
    # 获取鱼种名称列 (如果存在)
    fish_name_col = 'name' if 'name' in source_df.columns else 'Species'
    
    # 准备结果DataFrame
    result_rows = []
    
    # 为每个鱼种创建一行
    for idx, row in source_df.iterrows():
        # 开始构建新行
        new_row = {}
        
        # 添加ID和鱼名 (如果存在)
        if 'id' in source_df.columns:
            new_row['id'] = row['id']
        if fish_name_col in source_df.columns:
            new_row['name'] = row[fish_name_col]
        
        # 为每个结构体类型添加一对struct_type和coeff列
        for i, struct_type in enumerate(struct_columns):
            # 确定struct_type列和coeff列的名称
            type_col = f'struct_type{"" if i == 0 else f".{i}"}'  
            coeff_col = f'coeff{"" if i == 0 else f".{i}"}'
            
            # 将结构类型名称和系数值放入对应列
            new_row[type_col] = struct_type
            new_row[coeff_col] = float(row[struct_type])
        
        # 添加备注列 (如果存在)
        if 'mark' in source_df.columns:
            new_row['mark'] = row['mark']
        elif '备注' in source_df.columns:
            new_row['mark'] = row['备注']
        
        # 将新行添加到结果中
        result_rows.append(new_row)
    
    # 创建结果DataFrame
    result_df = pd.DataFrame(result_rows)
    return result_df

In [12]:
# 将OriginDf中的水下结构体数据转换为目标格式并保存

# 1. 从设计表中提取水下结构体列
struct_columns = segmentStruct  # 使用之前提取的结构体列名
print("水下结构体列名:", struct_columns)

# 2. 转换为结构化格式
structured_df = convert_wide_to_structured(OriginDf, struct_columns)

# 3. 检查结果
print("\n转换后的前5行:")
print(structured_df.head())

水下结构体列名: ['[水下结构体]开放水域', '[水下结构体]水草', '[水下结构体]石头', '[水下结构体]沉木', '[水下结构体]桥墩', '[水下结构体]深坑', '[水下结构体]尖脊', '[水下结构体]断层', '[水下结构体]岩架', '[水下结构体]湾子', '[水下结构体]泥底', '[水下结构体]碎石底']

转换后的前5行:
                  name  struct_type  coeff struct_type.1  coeff.1  \
0              Alewife  [水下结构体]开放水域    1.0     [水下结构体]水草      0.6   
1        Common_Shiner  [水下结构体]开放水域    0.0     [水下结构体]水草      1.0   
2        Green_Sunfish  [水下结构体]开放水域    0.6     [水下结构体]水草      1.0   
3  Pumpkinseed_Sunfish  [水下结构体]开放水域    1.0     [水下结构体]水草      1.0   
4     Bluegill_Sunfish  [水下结构体]开放水域    0.6     [水下结构体]水草      1.0   

  struct_type.2  coeff.2 struct_type.3  coeff.3 struct_type.4  ...  coeff.7  \
0     [水下结构体]石头      0.6     [水下结构体]沉木      0.6     [水下结构体]桥墩  ...      0.6   
1     [水下结构体]石头      0.6     [水下结构体]沉木      0.6     [水下结构体]桥墩  ...      0.6   
2     [水下结构体]石头      0.6     [水下结构体]沉木      0.0     [水下结构体]桥墩  ...      0.0   
3     [水下结构体]石头      1.0     [水下结构体]沉木      1.0     [水下结构体]桥墩  ...      1.0   
4     [水下结构

In [13]:
structSheet = AffinityPyxl['StructAffinity']

for r_idx, convertedRow in enumerate(dataframe_to_rows(structured_df, index=False, header=False)):
    targetRowName = "Cover_" + convertedRow[0]
    qualities = extract_qualities_for_fish(convertedRow[0])
    for q in qualities:
        rowInTopAffinitySheet = getFishNameRowIdxFromTopAffinitySheet(convertedRow[0],q)
        # print(f"RowInTopAffinitySheet : ", rowInTopAffinitySheet)
        if rowInTopAffinitySheet == -1:
            # print(f"没有找到鱼种 {originRow[0]} 的行名")
            continue
        # print(f"Row : ", convertedRow)

        topAffinitySheet.cell(row=rowInTopAffinitySheet, column=4, value=targetRowName)
    # print(targetRowName)
    headlessRow = convertedRow[1:-1] # 去掉行名和最后一列的备注
    # print(headlessRow)
    # 在第二列查找行名
    for cell in structSheet["B"]:
        if cell.value == targetRowName:
            # 找到行名，更新数据,更新的方法是将cell对应的行里，0、1元素不变，2（即第三个元素）号元素起，跟headlessRoww一样长的元素，使用headlessRow的值
            n = len(headlessRow)
            for i in range(n):
                structSheet.cell(row=cell.row, column=i+3, value=headlessRow[i])
            break
    else:
        # 如果没有找到行名，则添加新行, and keep the added row as a reference
        # 这里的row[1:]是去掉了行名的部分
        # 这一行的第一个数据是id，用上一行的id往上涨，涨到一个最近的能被10整除的数；
        # 也就是说，id是10的倍数
        maxRow = structSheet.max_row
        lastId = structSheet.cell(maxRow, 1).value
        if lastId is None:
            lastId = 2010000
        else:
            lastId = int(lastId)
        newId = lastId + 10 - (lastId % 10)
        # print(lastId)
        # add a new row, and keep the added row as a reference, the first cell is newId, the second cell is the row name, appended are headlessRow
        newRow = [newId, targetRowName] + list(headlessRow)
        structSheet.append(newRow)


# 觅食水层

In [14]:
waterLayerSegment = get_segment_by_start_name(startColumnNameWaterLayer)
print(waterLayerSegment)

['[地图水层]表层', '[地图水层]中层', '[地图水层]底层']


In [15]:
convertedDf = convert_wide_to_structured(OriginDf, waterLayerSegment)
# print("\n转换后的前5行:")
# print(convertedDf.head())

In [16]:
feedLayerSheet = AffinityPyxl["WaterLayerAffinity"]
colIdxFeedLayerInTopAffinitySheet = find_column_index(topAffinitySheet, "layer_id")

for convertedRow in dataframe_to_rows(convertedDf, index=False, header=False):
    headlessRow = convertedRow[1:-1]
    targetRowName = f"FeedLayer_{convertedRow[0]}"
    qualities = extract_qualities_for_fish(convertedRow[0])
    for q in qualities:
        rowIdx = getFishNameRowIdxFromTopAffinitySheet(convertedRow[0], q)
        if rowIdx and rowIdx > 0:
            cell = topAffinitySheet.cell(
                row=rowIdx,
                column=colIdxFeedLayerInTopAffinitySheet,
                value=targetRowName
            )
            cell.font = openpyxl.styles.Font(color="990099")
        else:
            raise ValueError(
                f"Cannot find top-affinity row for fish '{convertedRow[0]}' quality '{q}'"
            )
        
    rowFeedLayer = findRowByNameFromWolongSheet(feedLayerSheet, targetRowName)
    if rowFeedLayer is None:
        # create a new row in the feed layer sheet
        newRowIdx = feedLayerSheet.max_row + 1
        lastId = feedLayerSheet.cell(feedLayerSheet.max_row, 1).value
        if lastId is None:
            lastId = 2031000
        else:
            lastId = int(lastId)
        
        newId = lastId + 10 - lastId % 10
        newRow = [newId, targetRowName] + convertedRow[1:]
        for col in range(len(newRow)):
            feedLayerSheet.cell(
                row=newRowIdx,
                column=col+1,
                value=newRow[col]
            ).font = openpyxl.styles.Font(color="990099")
    else:
        # update the existing row in the feed layer sheet
        for col in range(len(headlessRow)):
            feedLayerSheet.cell(
                row=rowFeedLayer,
                column=col + 3,
                value=headlessRow[col]
            ).font = openpyxl.styles.Font(color="990099")

# 搞温度系数

In [17]:
tempSheet = AffinityPyxl["TempAffinity"]
tempColumnIdxInTopAffinitySheet = find_column_index(topAffinitySheet, "temp_id")

dictOriginCol2TempColIdx = {}
for k,v in dictTempColumns.items():
    columnIdx = find_column_index(tempSheet, v)
    if columnIdx is not None:
        dictOriginCol2TempColIdx[k] = columnIdx

OriginDf['中间温'] = OriginDf['中间温']*10

for species in OriginDf[columnNameFishSpecies]:
    # 在表格中查找这个名字
    qualities = extract_qualities_for_fish(species)
    for q in qualities:
        # 如果后续需要用整数作 key，再尝试转回 int
        try:
            quality_key = int(q)
        except ValueError:
            quality_key = q
        # 在表格中查找这个名字
        rowInTopAffinitySheet = getFishNameRowIdxFromTopAffinitySheet(species, quality_key)
        tempRowName = "Temp_" + species

        cell = topAffinitySheet.cell(row=rowInTopAffinitySheet,
                                     column=tempColumnIdxInTopAffinitySheet)
        cell.value = tempRowName
        cell.font = openpyxl.styles.Font(color="990099")

    # 在tempSheet中查找这个名字
    foundFishName = False
    for baitAffRow in range(2, tempSheet.max_row + 1):
        cell = tempSheet.cell(row=baitAffRow, column=2)
        if cell.value == tempRowName:
            foundFishName = True
            for key, value in dictOriginCol2TempColIdx.items():
                tempSheet.cell(row=baitAffRow, column=value, value=OriginDf.loc[OriginDf[columnNameFishSpecies] == species, key].values[0]).font=openpyxl.styles.Font(color="990099")
            break
    if not foundFishName:
        # add a new row TODO
        lastRow = tempSheet.max_row
        lastId = tempSheet.cell(row=lastRow, column=1).value
        if lastId is None:
            lastId = 2020000
        else:
            lastId = int(lastId)
        newId = lastId + 10 - (lastId % 10)
        # add a new row, and set the tempRowName in B column
        new_row_idx = tempSheet.max_row + 1
        # add a new row with green font formatting
        tempSheet.cell(row=new_row_idx, column=1, value=newId).font = openpyxl.styles.Font(color="990099")
        tempSheet.cell(row=new_row_idx, column=2, value=tempRowName).font = openpyxl.styles.Font(color="990099")
        for key, col_idx in dictOriginCol2TempColIdx.items():
            # fetch the origin value for this species and column
            val = OriginDf.loc[OriginDf[columnNameFishSpecies] == species, key].values[0]
            cell = tempSheet.cell(row=new_row_idx, column=col_idx, value=val)
            cell.font = openpyxl.styles.Font(color="990099")


# Baits & Lures

In [18]:
baitGroupStartId = 3000
baitGroupEndId = 4000
segmentStartStrBaits = "|真饵"
segmentStartStrLures = "|拟饵"
groupDefSheet = AffinityPyxl["EnvAffinityMap"]

segBaits = get_segment_by_start_name(segmentStartStrBaits)
segLures = get_segment_by_start_name(segmentStartStrLures)
if isinstance(segBaits, list):
    segBaits.insert(0, "Species")
else:
    segBaits = ["Species"] + (segBaits if segBaits is not None else [])
if isinstance(segLures, list):
    segLures.insert(0, "Species")
else:
    segLures = ["Species"] + (segLures if segLures is not None else [])

print(segBaits)

['Species', '[真饵]种子', '[真饵]昆虫', '[真饵]甲壳', '[真饵]鱼饵', '[真饵]鱼卵', '[真饵]面团', '[真饵]谷物', '[真饵]肉饵', '[真饵]乳制']


In [19]:
baitsDf = OriginDf[segBaits]
# check how many dupilicates are for each row of baitsDf
duplicateCounts = baitsDf.groupby(baitsDf.columns.tolist()[1:]).size().reset_index().rename(columns={0:'count'})
# print(duplicateCounts)
baitGroupSheet = AffinityPyxl["BaitTypeAffinity"]
mapWorksheet = AffinityPyxl["EnvAffinityMap"]

In [20]:
def add_bait_group_if_not_exists(baitGroup):
    colIdxId = find_column_index(mapWorksheet, "id")
    colIdxName = find_column_index(mapWorksheet, "name")
    # only look at rows that has id between baitGroupStartId and baitGroupEndId
    maxIdBaitGroups = baitGroupStartId
    foundBaitGroup = False
    for row in mapWorksheet.iter_rows(min_row=5, max_row=mapWorksheet.max_row, min_col=colIdxId, max_col=colIdxName):
        if row[1].value == baitGroup:
            foundBaitGroup = True
            break
        id = row[0].value
        try:
            id = int(id)
        except(ValueError, TypeError):
            print(f"Warning: Could not convert ID '{id}' to int for row. Skipping ID check for this row.")
            continue

        # id = int(id)
        if id is not None and id >= baitGroupStartId and id <= baitGroupEndId:
            if id > maxIdBaitGroups:
                maxIdBaitGroups = id
                
    if not foundBaitGroup:
        newRowIdx = mapWorksheet.max_row + 1
        maxIdBaitGroups += 1
        cell_id = mapWorksheet.cell(row=newRowIdx, column=colIdxId)
        cell_id.value = maxIdBaitGroups
        cell_id.font = Font(color="990099")
        
        cell_name = mapWorksheet.cell(row=newRowIdx, column=colIdxName)
        cell_name.value = baitGroup
        cell_name.font = Font(color="990099")


In [21]:
colIdxId = find_column_index(baitGroupSheet, "id")
colIdxCoeffGroup = find_column_index(baitGroupSheet, "bait_type_coeff_group")
colIdxBaitType = find_column_index(baitGroupSheet, "bait_sub_type")
colIdxCoeff = find_column_index(baitGroupSheet, "coeff")
colIdxPoseGroup = find_column_index(baitGroupSheet, "pose_group")
POSEBP_GENERAL_REALBAIT = "posebp_general_realbait"

for baitAffRow in dataframe_to_rows(baitsDf, index=False, header=False):
    # print(row)
    fishSpecies = baitAffRow[0]
    # postfixes = get_quality_strings_for_fishname(fishSpecies)
    qualities = extract_qualities_for_fish(fishSpecies)
    groupName = "BaitGroup_" + fishSpecies
    add_bait_group_if_not_exists(groupName)
    # update bait group name within top affinity sheet
    for quality in qualities:
        # print(fishSpecies + postfix)
        fishQuality = fishSpecies + get_qualitystr_from_qualityidx(quality)
        topAffRowIdx = getFishNameRowIdxFromTopAffinitySheet(fishSpecies, quality)
        if topAffRowIdx == -1:
            print("Could not find top affinity row for " + fishQuality)
            continue
        colIdx = find_column_index(topAffinitySheet, "bait_type_coeff_group")
        topAffinitySheet.cell(row=topAffRowIdx, column=colIdx, value = groupName).font = openpyxl.styles.Font(color="990099")
        
    # create or update data within bait group sheet
    # first, try to find groupName in baitGroupSheet, column is B

    for baitType in segBaits[1:]:
        foundExistingRow = False
        # try find row with groupName(column index = colIdxCoeffGroup) and baitType(column index = colIdxBaitType)
        for rowIdx in range(5, baitGroupSheet.max_row + 1):
            if baitGroupSheet.cell(row=rowIdx, column=colIdxCoeffGroup).value == groupName and baitGroupSheet.cell(row=rowIdx, column=colIdxBaitType).value == baitType:
                # update row with groupName and baitType
                baitGroupSheet.cell(row=rowIdx, column=colIdxCoeff, value = baitAffRow[segBaits.index(baitType)]).font = openpyxl.styles.Font(color="990099")
                foundExistingRow = True
                break
        if not foundExistingRow:
            # create new row with groupName and baitType
            lastId = baitGroupSheet.cell(row=baitGroupSheet.max_row, column=colIdxId).value
            lastId = int(lastId)
            newId = lastId + 10 - (lastId % 10)
            baitGroupSheet.cell(row=baitGroupSheet.max_row + 1, column=colIdxId, value = newId).font = openpyxl.styles.Font(color="990099")
            baitGroupSheet.cell(row=baitGroupSheet.max_row, column=colIdxCoeffGroup, value = groupName).font = openpyxl.styles.Font(color="990099")
            baitGroupSheet.cell(row=baitGroupSheet.max_row, column=colIdxBaitType, value = baitType).font = openpyxl.styles.Font(color="990099")
            baitGroupSheet.cell(row=baitGroupSheet.max_row, column=colIdxCoeff, value = baitAffRow[segBaits.index(baitType)]).font = openpyxl.styles.Font(color="990099")
            baitGroupSheet.cell(row=baitGroupSheet.max_row, column=colIdxPoseGroup, value = POSEBP_GENERAL_REALBAIT).font = openpyxl.styles.Font(color="990099")
            
        


## 开始弄lures

In [22]:
lureDf = OriginDf[segLures]
poseBpNameDf = pd.read_excel(OriginWorkbookPath, sheet_name="拟饵-姿态", index_col=0)
# print(poseBpNameDf.head())
# 先把poseBpNameDf 转成字典
dictDefaultPoseBp = poseBpNameDf.to_dict()['默认pose系数组']
print(dictDefaultPoseBp)


# 用 ”[拟饵]T尾“来找对应的”默认pose系数组“值
print(dictDefaultPoseBp["[拟饵]T尾"])


{'[拟饵]T尾': 'posebp_lure_t-tail', '[拟饵]卷尾': 'posebp_lure_grub', '[拟饵]软虫': 'posebp_lure_softworm', '[拟饵]虾管': 'posebp_lure_tube_shrimp', '[拟饵]米诺': 'posebp_lure_minnow', '[拟饵]波爬': 'posebp_lure_wobbler', '[拟饵]勺子亮片': 'posebp_lure_spoon', '[拟饵]旋转亮片': 'posebp_lure_spinner', '[拟饵]VIB': 'posebp_lure_vib', '[拟饵]水面拖拉机': 'posebp_lure_topwater', '[拟饵]铅笔': 'posebp_lure_pencil', '[拟饵]嘈杂饵': 'posebp_lure_rattle', '[拟饵]多节鱼': 'posebp_lure_jointed', '[拟饵]雷蛙': 'posebp_lure_frog', '[拟饵]胡须佬': 'posebp_lure_whisker', '[拟饵]复合亮片': 'posebp_lure_comp_spinner', '[拟饵]摇滚饵': 'posebp_lure_rockbait'}
posebp_lure_t-tail


In [23]:


for lureAffRow in dataframe_to_rows(lureDf,index=False, header=False):
    # print(row)
    fishSpecies = lureAffRow[0]    # print(row)
    # postfixes = get_quality_strings_for_fishname(fishSpecies)
    qualities = extract_qualities_for_fish(fishSpecies)
    groupName = "BaitGroup_" + fishSpecies
    add_bait_group_if_not_exists(groupName)
    
    # update bait group name within top affinity sheet
    for quality in qualities:
        # print(fishSpecies + postfix)
        fishQuality = fishSpecies + get_qualitystr_from_qualityidx(quality)
        topAffRowIdx = getFishNameRowIdxFromTopAffinitySheet(fishSpecies, quality)
        if topAffRowIdx == -1:
            print("Could not find top affinity row for " + fishQuality)
            continue
        colIdx = find_column_index(topAffinitySheet, "bait_type_coeff_group")
        topAffinitySheet.cell(row=topAffRowIdx, column=colIdx, value = groupName).font = openpyxl.styles.Font(color="990099")
        
    # create or update data within bait group sheet
    # first, try to find groupName in baitGroupSheet, column is B

    for lureType in segLures[1:]:
        defaultPoseGroupName = dictDefaultPoseBp[lureType]
        foundExistingRow = False
        # try find row with groupName(column index = colIdxCoeffGroup) and lureType(column index = colIdxBaitType)
        for rowIdx in range(5, baitGroupSheet.max_row + 1):
            if baitGroupSheet.cell(row=rowIdx, column=colIdxCoeffGroup).value == groupName and baitGroupSheet.cell(row=rowIdx, column=colIdxBaitType).value == lureType:
                # update row with groupName and lureType
                baitGroupSheet.cell(row=rowIdx, column=colIdxCoeff, value = lureAffRow[segLures.index(lureType)]).font = openpyxl.styles.Font(color="990099")
                foundExistingRow = True
                break
        if not foundExistingRow:
            # create new row with groupName and lureType
            lastId = baitGroupSheet.cell(row=baitGroupSheet.max_row, column=colIdxId).value
            lastId = int(lastId)
            newId = lastId + 10 - (lastId % 10)
            baitGroupSheet.cell(row=baitGroupSheet.max_row + 1, column=colIdxId, value = newId).font = openpyxl.styles.Font(color="990099")
            baitGroupSheet.cell(row=baitGroupSheet.max_row, column=colIdxCoeffGroup, value = groupName).font = openpyxl.styles.Font(color="990099")
            baitGroupSheet.cell(row=baitGroupSheet.max_row, column=colIdxBaitType, value = lureType).font = openpyxl.styles.Font(color="990099")
            baitGroupSheet.cell(row=baitGroupSheet.max_row, column=colIdxCoeff, value = lureAffRow[segLures.index(lureType)]).font = openpyxl.styles.Font(color="990099")
            baitGroupSheet.cell(row=baitGroupSheet.max_row, column=colIdxPoseGroup, value = defaultPoseGroupName).font = openpyxl.styles.Font(color="990099")
    

# 投鱼 & 体长

In [24]:
# OriginDf.head()
colDataTierWholeRndWeight = OriginDf["阶内权重"]
colDataQualitiesInTier = OriginDf["品质"]
colDataPond = OriginDf["地图"]
colDataSpecies = OriginDf["Species"]
colDataTier = OriginDf["等阶"]

pondWorkbook = openpyxl.open(PondWorkbookPath)
sheetFishRelease = pondWorkbook["FishRelease"]

# 创建常量，存储这几个列名：id	stock_id	fish_id	fish_num	prob_weight_ideal	min_env_coeff	min_adapt_coeff	length_min	length_max	is_reduce
# 定义8个列名常量
COL_ID = "id"
COL_STOCK_ID = "stock_id" 
COL_FISH_ID = "fish_id"
COL_FISH_NUM = "fish_num"
COL_PROB_WEIGHT = "prob_weight_ideal"
COL_MIN_ENV = "min_env_coeff"
COL_MIN_ADAPT = "min_adapt_coeff"
COL_LENGTH_MIN = "length_min"
COL_LENGTH_MAX = "length_max" 
COL_IS_REDUCE = "is_reduce"

# 分别获取列索引
col_idx_id = find_column_index(sheetFishRelease, COL_ID)
col_idx_stock = find_column_index(sheetFishRelease, COL_STOCK_ID)
col_idx_fish = find_column_index(sheetFishRelease, COL_FISH_ID)
col_idx_num = find_column_index(sheetFishRelease, COL_FISH_NUM)
col_idx_prob = find_column_index(sheetFishRelease, COL_PROB_WEIGHT)
col_idx_env = find_column_index(sheetFishRelease, COL_MIN_ENV)
col_idx_adapt = find_column_index(sheetFishRelease, COL_MIN_ADAPT)
col_idx_len_min = find_column_index(sheetFishRelease, COL_LENGTH_MIN)
col_idx_len_max = find_column_index(sheetFishRelease, COL_LENGTH_MAX)
col_idx_reduce = find_column_index(sheetFishRelease, COL_IS_REDUCE)

idStartPond0 = 300100
idIntervalEachPond = 100

# 定义几个配置常量字符串
NOT_REDUCE = "[表达式]否"
# print(colDataPond)


In [25]:
# 从dictQualityDf构建品质配置字典
QUALITY_CONFIG = {}
for _, row in dictQualityDf.iterrows():
    try:
        quality_id = int(row["品质"])
        QUALITY_CONFIG[quality_id] = {
            "suffix": row["后缀"].strip(),
            "enum": row["枚举"].strip(),
            "weight": int(row["权重占比"])
        }
    except (ValueError, KeyError) as e:
        print(f"品质字典第{_+1}行数据异常: {e}")

sheetNameFishLength = "体长"
fishLenDf = pd.read_excel(OriginWorkbookPath, sheet_name=sheetNameFishLength)
# 体长表格列名常量
ColNameMinLength = "MinLength"
ColNameMaxLength = "MaxLength"
ColNamePondIdx = "Lake"
ColNameFishNameWithQuality = "NameWithQuality"

def getMinMaxLenForFishWithQuality(fishNameWithQuality:str, pondIdx:int)->(int,int):
    # 使用&运算符连接两个条件进行数据筛选
    row = fishLenDf[(fishLenDf[ColNameFishNameWithQuality] == fishNameWithQuality) & (fishLenDf[ColNamePondIdx] == pondIdx)]
    minLen = row[ColNameMinLength]  # 使用iloc[0]获取第一个匹配结果
    maxLen = row[ColNameMaxLength]
    # print("minLen", minLen, "maxLen", maxLen)
    minLen = int(minLen)
    maxLen = int(maxLen)
    return minLen,maxLen

In [28]:
# 对每一行originDf 1 按品质拆分权重  2 将权重填入sheetFishRelease
for rowIdx in range(OriginDf.shape[0]):
    # print(rowIdx)
    species = colDataSpecies[rowIdx]
    qualitiesInTier = extract_qualities_for_fish(species)
    qualitiesCount = len(qualitiesInTier)
    wholeRndWeight = colDataTierWholeRndWeight[rowIdx]
    total_weight= sum(QUALITY_CONFIG[q]["weight"] for q in qualitiesInTier)
    allocated_weights = []
    pondId = colDataPond[rowIdx]
    stockName = f"FishReleasePond_{pondId}"

    for qualityIdx in qualitiesInTier:
        ratio = QUALITY_CONFIG[qualityIdx]["weight"] / total_weight
        allocated = int(wholeRndWeight * ratio)

        postfix = get_qualitystr_from_qualityidx(qualityidx=qualityIdx)
        fishQualityRowName = "Fish_" + species + postfix
        
        minLen, maxLen = getMinMaxLenForFishWithQuality(fishQualityRowName, pondId)

        # 看sheetFishRelease里是否有对应的配置，对应的条件是： 
        foundExistingFishRelease = False
        for row in sheetFishRelease.iter_rows(min_row=5, max_row=sheetFishRelease.max_row, min_col=1, max_col=sheetFishRelease.max_column):
            # print("row[col_idx_stock]",row[col_idx_stock])
            # print("row[col_idx_stock].value", row[col_idx_stock].value)
            if (str(row[col_idx_stock - 1].value).strip() == str(stockName).strip()) and (str(row[col_idx_fish - 1].value).strip() == str(fishQualityRowName).strip()):                # 修改为通过row元素直接赋值
                row[col_idx_prob - 1].value = allocated
                row[col_idx_prob - 1].font = Font(color="990099")
                row[col_idx_num - 1].value = allocated 
                row[col_idx_num - 1].font = Font(color="990099")
                row[col_idx_len_min - 1].value = minLen
                row[col_idx_len_min - 1].font = Font(color="990099")
                row[col_idx_len_max - 1].value = maxLen
                row[col_idx_len_max - 1].font = Font(color="990099")
                foundExistingFishRelease = True
                break
        if not foundExistingFishRelease:
            # create new row
            maxRowIdx = sheetFishRelease.max_row
            newRowIdx = maxRowIdx + 1
            
            # 设置各列的值（根据实际需求调整字段值）
            sheetFishRelease.cell(row=newRowIdx, column=col_idx_stock, value=stockName).font = Font(color="990099")
            sheetFishRelease.cell(row=newRowIdx, column=col_idx_fish, value=fishQualityRowName).font = Font(color="990099")
            sheetFishRelease.cell(row=newRowIdx, column=col_idx_prob, value=allocated).font = Font(color="990099")
            sheetFishRelease.cell(row=newRowIdx, column=col_idx_num, value=allocated).font = Font(color="990099")
            
            # 设置其他列的默认值（根据实际需求调整）
            sheetFishRelease.cell(row=newRowIdx, column=col_idx_env, value=0).font = Font(color="990099")  # 示例默认值
            sheetFishRelease.cell(row=newRowIdx, column=col_idx_adapt, value=0).font = Font(color="990099")
            sheetFishRelease.cell(row=newRowIdx, column=col_idx_len_min, value=qualityIdx*10+5).font = Font(color="990099")
            sheetFishRelease.cell(row=newRowIdx, column=col_idx_len_max, value=qualityIdx*10+15).font = Font(color="990099")
            sheetFishRelease.cell(row=newRowIdx, column=col_idx_reduce, value=NOT_REDUCE).font = Font(color="990099")
            
            # ID字段需要根据实际生成规则设置（示例为自动递增）
            sheetFishRelease.cell(row=newRowIdx, column=col_idx_id, value=maxRowIdx+idStartPond0).font = Font(color="990099")
            
            print(f"新建行 {newRowIdx}: {stockName}/{fishQualityRowName}")


C:\Users\futou\AppData\Local\Temp\ipykernel_35284\3695566328.py:28: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  minLen = int(minLen)
C:\Users\futou\AppData\Local\Temp\ipykernel_35284\3695566328.py:29: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  maxLen = int(maxLen)
C:\Users\futou\AppData\Local\Temp\ipykernel_35284\3695566328.py:28: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  minLen = int(minLen)
C:\Users\futou\AppData\Local\Temp\ipykernel_35284\3695566328.py:29: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  maxLen = int(maxLen)
C:\Users\futou\AppData\Local\Temp\ipykernel_35284\3695566328.py:28: FutureWarnin

TypeError: cannot convert the series to <class 'int'>

# 将钓场鱼品质列表写入鱼单价工具

In [ ]:
PriceToolWorkbook = openpyxl.open(PriceToolWorkbookPath)
sheetQualitiesList = PriceToolWorkbook["钓场鱼品质列表"]

# empty this sheet first
if sheetQualitiesList.max_row > 1:  # Check if there are rows beyond header
    sheetQualitiesList.delete_rows(2, sheetQualitiesList.max_row)  # Delete from row 2 to end

# fill in header values(first row), 值为 鱼种	品质	钓场，并将字体设为bold
# 填充表头（第一行）并设置粗体
header_row = sheetQualitiesList[1]  # 第一行
header_row[0].value = "鱼种"
header_row[0].font = Font(bold=True)
header_row[1].value = "品质" 
header_row[1].font = Font(bold=True)
header_row[2].value = "钓场"
header_row[2].font = Font(bold=True)
header_row[3].value = "等阶"  # 原第四列改为等阶
header_row[3].font = Font(bold=True)
# 在表头部分新增随机权重列
header_row[4].value = "随机权重"  # 原第五列改为随机权重
header_row[4].font = Font(bold=True)
header_row[5].value = "最小长度"  # 后续列顺延
header_row[5].font = Font(bold=True)
header_row[6].value = "最大长度"
header_row[6].font = Font(bold=True)
# header_row[5].value = "备注"
# header_row[5].font = Font(bold=True)
curRowIdx = 1

for rowIdx in range(OriginDf.shape[0]):
    # print(rowIdx)
    species = colDataSpecies[rowIdx]
    qualitiesInTier = extract_qualities_for_fish(species)
    qualitiesCount = len(qualitiesInTier)
    wholeRndWeight = colDataTierWholeRndWeight[rowIdx]
    eachRndWeight = int(wholeRndWeight/qualitiesCount)
    pondId = colDataPond[rowIdx]
    stockName = f"FishReleasePond_{pondId}"
    total_weight= sum(QUALITY_CONFIG[q]["weight"] for q in qualitiesInTier)


    for q in qualitiesInTier:
        curRowIdx = curRowIdx + 1
        qualityStr = get_qualitystr_from_qualityidx(q)
        fishQualityRowName = "Fish_" + species + qualityStr
        ratio = QUALITY_CONFIG[q]["weight"] / total_weight
        allocated = int(wholeRndWeight * ratio)

        # fill in values for 鱼种、品种、钓场
        sheetQualitiesList.cell(row=curRowIdx, column=1, value=species).font = Font(color="990099")
        sheetQualitiesList.cell(row=curRowIdx, column=2, value=qualityStr).font = Font(color="990099")
        sheetQualitiesList.cell(row=curRowIdx, column=3, value=pondId).font = Font(color="990099")
        sheetQualitiesList.cell(row=curRowIdx, column=4, value=colDataTier[rowIdx]).font = Font(color="990099")
        sheetQualitiesList.cell(row=curRowIdx, column=5, value=allocated).font = Font(color="990099")

PriceToolWorkbook.save(PriceToolWorkbookPath)
PriceToolWorkbook.close()

# 气压敏感 & 饵长 & 时段

In [ ]:
# 创建列名映射字典
COLUMN_MAPPING = {
    "min_adapt_lure_ratio": "min_adapt_lure_ratio",
    "max_adapt_lure_ratio": "max_adapt_lure_ratio",
    "under_length_decay_coeff": "under_length_decay_coeff",
    "over_length_decay_coeff": "over_length_decay_coeff",
    "max_accept_length_ratio": "max_accept_length_ratio",
    "气压敏感度": "pressure_sensitivity",  # 特殊处理中文列名
    "时段习性组": "period_coeff_group"
}

for idx, originRow in OriginDf.iterrows():
    species = originRow[columnNameFishSpecies]
    qualities = extract_qualities_for_fish(species)
    for q in qualities:
        topAffRowIdx = getFishNameRowIdxFromTopAffinitySheet(species, q)
        # 遍历映射字典的键值对
        for k, v in COLUMN_MAPPING.items():
            orginValue = originRow[k]
            targetColumnIdx = find_column_index(topAffinitySheet, v)
            # 设置单元格值
            topAffinitySheet.cell(row=topAffRowIdx, column=targetColumnIdx,value=orginValue).font = Font(color="990099")


# 体长

In [ ]:
# 已在投鱼部分处理

# 单价

In [ ]:
priceDf = pd.read_excel(PriceToolWorkbookPath, sheet_name="鱼单价工具", header=1)
# drop row 1 and row 3
priceDf = priceDf.drop([0, ])
priceDf = priceDf.reset_index(drop=True)


In [ ]:

# priceDf.head()



In [ ]:
# 定义列映射关系
COLUMN_MAPPING = {
    "单价": "basic_reward_num",
    "xp": "exp"
}
# 创建列索引缓存字典
target_col_indices = {}
header_row = fishQualitySheet[2]
for src_col, target_col in COLUMN_MAPPING.items():
    for idx, cell in enumerate(header_row, 1):  # openpyxl列号从1开始
        if cell.value == target_col:
            target_col_indices[src_col] = idx
            break
    else:
        print(f"警告: 目标列'{target_col}'未找到，已跳过")

for idx, price_row in priceDf.iterrows():
    try:
        # 获取鱼种标识
        fishSpecies = price_row["鱼种"] 
        quality = price_row["稀有度"]
        fishQualityRowName = fishSpecies + quality
        
        # 在quality表中查找对应行（假设通过B列匹配）
        found = False
        for q_row in fishQualitySheet.iter_rows(min_row=4):
            if q_row[1].value == fishQualityRowName:
                for src_col, target_col in COLUMN_MAPPING.items():
                    if src_col not in target_col_indices:
                        continue
                    col_idx = target_col_indices[src_col]
                    q_row[col_idx - 1].value = price_row[src_col]
                    q_row[col_idx - 1].font = Font(color="990099")
                found = True
                break
        
        if not found:
            print(f"警告: 未找到鱼种'{fishQualityRowName}'的品质配置，已跳过")
            
    except KeyError as e:
        print(f"行{idx}数据异常，缺少必要字段: {e}")

# 保存&清理

In [ ]:
FishBasicPyxl.save(FishBasicWorkbookPath)

AffinityPyxl.save(AffinityWorkbookPath)
pondWorkbook.save(PondWorkbookPath)

# AffinityPyxl.save("StructAffinity")
AffinityPyxl.close()
FishBasicPyxl.close()
pondWorkbook.close()